<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/01_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-

import requests
import json

from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup
import time

import pandas as pd
import numpy as np

아래 URL 에서 각 매장의 리뷰 정보를 제공하는 JSON 을 얻을 수 있음을 확인  
id 에 각 매장에 해당하는 고유번호를 넣고, count = 300 으로 설정하여 총 167개의 매장마다 최대 300개의 리뷰를 수집하도록함

In [ ]:
id = 537793
url = f"https://www.yogiyo.co.kr/api/v1/reviews/{id}/?count=300&only_photo_review=false&page=1&sort=time"
url

'https://www.yogiyo.co.kr/api/v1/reviews/537793/?count=300&only_photo_review=false&page=1&sort=time'

위 JSON 데이터에서 "comment", "rating", "rating_taste", "rating_quantity", "owner_reply" 등의 정보를 수집

In [ ]:
ids = [537793, 314150, 313900, 1114060, 1200475, 1044404, 1018503, 1034857, 556398, 475929, 559016, 338597, 300758, 354386, 1186009, 41897, 304453, 1038734, 1209707, 1047264, 86995, 346911, 478997, 462085, 263472, 253327, 472492, 411554, 370695, 1049984, 1181946, 1060040, 398365, 1016855, 1086501, 545150, 274677, 1037295, 360741, 412393, 240496, 1053480, 269030, 490487, 495643, 557742, 580769, 1002383, 585251, 450790, 303449, 1023351, 1203455, 62248, 450812, 1104123, 1183921, 356094, 445494, 1128582, 401263, 1053620, 1143182, 1195581, 1210746, 1145207, 1181786, 1175720, 1218230, 1113166, 1151723, 1051569, 1166284, 1185234, 1012700, 405781, 587999, 471576, 1079453, 1172190, 1190471, 434287, 1057229, 364086, 265049, 396589, 557515, 1105987, 296219, 1075218, 354989, 1111805, 1024428, 1052084, 63915, 1054297, 1225071, 1007093, 1025067, 407758, 571677, 1151433, 1131181, 264772, 1106348, 1134693, 489729, 1075182, 501002, 391355, 562835, 1139428, 1029995, 1195566, 583035, 1184578, 1209522, 530807, 1203243, 483553, 562647, 517839, 439888, 308210, 1068695, 384416, 410902, 447787, 582191, 490577, 346156, 1023306, 1002641, 1222581, 389930, 427854, 468817, 534266, 516034, 1135524, 16761, 318354, 1170762, 1211043, 1205772, 528403, 1173254, 327058, 435065, 533344, 1188119, 1092944, 310865, 1016160, 496420, 235211, 450724, 1094403, 247643, 318823, 1146747, 1178593, 443708, 1195229, 264943, 296138, 1186975]
stores = ["1988응답하라추억의옛날도시락-방이점", "워커스커피 로스터스", "떡깨비-가락점", "당치땡-헬리오시티점", "우시장국밥-석촌점", "고돼지-송파점", "마라의신마라탕&마라샹궈", "춘리마라탕-송파점", "손수경의육회담은연어-가락점", "무국적식탁-송파점", "청년치킨-삼전점", "송파만다린", "배달의쌀국수-송파점", "닭강점-헬리오시티점", "토핑폭탄김치찜&김치찌개-송파점", "조형훈족발보쌈", "여왕의브런치", "모어댄버터", "플러스82-송파점", "더티베지", "잭아저씨족발보쌈-본점", "육회&연어-미아본점", "갈비민족-본점", "구구족-성신여대역점", "피자보이시나-대학로점", "왕빈자삼파전-미아본점", "행복한찜닭-성북점", "스시사소우", "저팔계&중국&마라탕&덮밥&요리&전문점", "명작파스타-성북점", "올데이파스타-동대문구점", "대한냉면-성북점", "인생제육", "고기듬뿍미트박스-성북점", "피자스웨그-성북점", "청춘식당묵은지김치찜", "삽교원조두리곱창", "뜸들이다-북서울꿈의숲점", "더팔당매운오징어&닭발 김치찜", "우리집반찬도시락-혜화막내딸지점", "수피자", "마라쿡", "김포불닭발", "헬로팬닭갈비&마약볶음밥", "대찌 대파생고기김치찌개&정육왕국물두루치기", "구래상회", "승도리네곱도리탕", "자성당", "수락", "나인곱창", "어시장", "돈까스회관", "호랑이초밥", "땡초곱창막창", "피자이탈리", "텐텐마라탕", "과일에반하다", "삼다수", "병천토속순대", "최고남제육명가", "짱닭치킨", "걸작떡볶이", "직화삼겹직구삼", "짜글이가돼었소", "부대찌개자신있는집", "토핑폭탄김치찜&김치찌개-부천점", "1989마라탕-부천점", "미친피자-부평점", "천년닭강정-부천부평점", "수상한삼겹살-부천점", "수제죽전문점-본가진죽-본점", "오늘은 분식", "이끌림마라탕", "탐나는피자-부평점", "고기혁명-부천점", "라화방마라탕-신중동점", "전금례닭볶음탕-부천점", "큰아들백순대", "공주닭발-부천시청점", "신자매김치찜&김치찌개", "짱닭치킨-도촌점", "이태리면가게-야탑점", "폴트버거-판교점", "랜돌프뉴욕페페로니-야탑점", "홍싸롱-수제돈까스&파스타", "쇼부덮밥", "카산도-정자본점", "부성초밥-미금본점", "닭장수섭삼계탕&전기구이통닭", "깨돌이김밥-미금점", "울진죽변항", "GTS버거-분당정자점", "코브라독스-분당미금역점", "마라하오-죽전점", "땅끝수산", "초밥대통령-용인수지점", "올데이케밥&샐러드", "야키토리 카마쿠라", "비오키친-서현본점", "부산오빠1인국밥-분당점", "알촌", "밈피자", "고씨네카레", "홍스족발", "뜸들이다", "감성커피", "짚신스시", "떡형", "39도시락", "피제이피자", "황궁쟁반짜장", "OK반점", "모모타코야키&카페", "꼬알라파이", "진지덮밥", "피자프라텔로", "한식세끼1인김치찜&김치찌개", "윤희횟집", "시민보쌈족발&감자탕", "카페봄봄", "킹프레소빅와플", "최가네한쌈", "창타이누들", "보돌미역", "홍대개미 ", "7번가피자-용암점", "이배달 마약삼겹떡볶이 상당구점", "스시리", "봉명동내커피-대전4호점", "미락칼국수-신탄진점", "이화수육개장-신일점", "팔도한마당순대", "국수나무-목동충촌점", "도우라이크피자-대전동구점", "이토우야-우송대점", "빽다방-대전역점", "고기듬뿍대왕비빔밥-대흥문창점", "고기잔뜩짜글이찌개&부대찌개", "혼밥대왕-서대전점", "순살만공격-유성봉명점", "본점 장충왕족발보쌈-괴정동", "써브웨이-대전 관평점", "맘스터치 대덕테크노벨리점", "덮덮밥-대전전민점", "탕화쿵푸 마라탕 가오점", "빅스타피자-대전한남대점", "손이가시래기감자탕", "안녕,닭-충남대점", "상무초밥-유성점", "롱쓰부대찌개-광주백운점", "도야짬뽕 계림점", "냉면&숯불고기전문점줄벼락-조대점", "후스폭립", "인생아구찜-광주본점", "빕스얌딜리버리-광주광천점", "놀부보쌈족발-북구(전대)점", "띵하오차이", "유쾌한꼬막&한끼끝판왕-광주직영점", "놀부 부대찌개-광주우산점", "박가부대-전주평화점", "용문", "백소정-객사점", "콩샌전북대점", "프레드피자-전주1호점", "원할머니보쌈-전주평화점", "누보닭갈비", "짬뽕관-전주금암점", ]

len(ids), len(stores)

(167, 167)

In [ ]:
review_lst = []

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
driver = webdriver.Chrome("chromedriver-win32\chromedriver.exe")

for id, name in zip(ids, stores):
    url = f"https://www.yogiyo.co.kr/api/v1/reviews/{id}/?count=300&only_photo_review=false&page=1&sort=time"
    driver.get(url)
    driver.implicitly_wait(5)
    h = driver.page_source
    w = driver.find_element(By.TAG_NAME, "pre").text
    driver.implicitly_wait(5)
    data = json.loads(w)
    time.sleep(5)
    driver.quit()


    for d in data:
        menus = []
        if d.get("menu_items") is not None :
            menus = [ menu.get("name") for menu in d.get("menu_items") ]

        if d.get("owner_reply") is not None :

            review_lst.append({

                "가게이름" : name,
                "고객리뷰" :d["comment"],
                "별점" : d["rating"],
                "맛별점" : d["rating_taste"],
                "양별점": d["rating_quantity"],
                "배달별점" : d["rating_delivery"] ,
                "사장댓글": d["owner_reply"]["comment"],
                "메뉴" : ",".join(menus)
            })




C:\Users\USER\AppData\Local\Temp\ipykernel_1600\574062276.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver-win32\chromedriver.exe")


In [ ]:
df = pd.DataFrame(review_lst)
df

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴
0,1988응답하라추억의옛날도시락-방이점,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,5,5.0,5.0,5.0,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,[직화]삼겹옛날도시락
1,1988응답하라추억의옛날도시락-방이점,기름 있는 부분이 생각보다 많아요! 참고하세요:),5,5.0,5.0,5.0,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,"[직화]삼겹옛날도시락,볶음김치"
2,1988응답하라추억의옛날도시락-방이점,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,5,5.0,5.0,5.0,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,"[직화]삼겹옛날도시락,우삼겹청국장찌개＋대접밥"
3,1988응답하라추억의옛날도시락-방이점,"..,........",5,5.0,5.0,5.0,저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,삼겹정찬도시락
4,1988응답하라추억의옛날도시락-방이점,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,5,5.0,4.0,5.0,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,"갈비옛날도시락＋미니냉면＋（서비스）콜라,스팸옛날도시락＋김찌 정식＋（서비스）콜라,부드러..."
...,...,...,...,...,...,...,...,...
37135,짬뽕관-전주금암점,맛있어요 ㅎ저번에주문하고 맛있어서또주문핰건데\n늦은시간에시켜서그렄지 국물이 엄청 걸...,5,5.0,5.0,5.0,고객님 안녕하세요 짬뽕관입니다~~^-^\n소중한 리뷰 너무 감사합니다!!🙂\n재주문...,짬뽕 ＋ 탕수육 set
37136,짬뽕관-전주금암점,맛있어요 또시킬게요,5,5.0,5.0,5.0,고객님 안녕하요 짬뽕관입니다!!^_^\n소중한 리뷰 남겨주셔 서 정말 달감사합니다!...,순두부짬뽕
37137,짬뽕관-전주금암점,정말 맛있게 잘먹었어요!!자주 시켜 먹을께요!,5,5.0,5.0,5.0,고객님 안녕하세요 짬뽕관입니다~~^_^\n맛있게 드셨다는 말씀에 힘이 나는거 같습니...,콩국수
37138,짬뽕관-전주금암점,와우.. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 ...,5,5.0,5.0,5.0,고객님 안녕하세요 짬뽕관입니다!!^_^\n맛있게 드셔주시고 정성스러운 사진 리뷰까지...,"차슈짬뽕,마파두부밥"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37140 entries, 0 to 37139
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   가게이름    37140 non-null  object 
 1   고객리뷰    37140 non-null  object 
 2   별점      37140 non-null  int64  
 3   맛별점     37140 non-null  float64
 4   양별점     37140 non-null  float64
 5   배달별점    37140 non-null  float64
 6   사장댓글    37140 non-null  object 
 7   메뉴      37140 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 2.3+ MB


In [ ]:
df.to_csv("yogiyo_reviews_0908.csv", index = False)